In [53]:
# map_url = 'https://nominatim.openstreetmap.org/reverse?format=json'

# first_loc_params = {
#     'lat': coords1[0],
#     'lon': coords1[1]
# }
# second_loc_params = {
#     'lat': coords2[0],
#     'lon': coords2[1]
# }

# response_1 = requests.get(map_url, params=first_loc_params, headers=headers, timeout=10)
# response_2 = requests.get(map_url, params=second_loc_params, headers=headers, timeout=10)

# if response_1.status_code == 200 and response_2.status_code == 200:
#     data_1 = response_1.json()
#     data_2 = response_2.json()
# else:
#     print(f"Error: {response_1.status_code}, {response_1.reason}")
#     exit()

# location_1 = data_1[0]
# location_2 = data_2[0]

# pprint(data_1)
# pprint(data_2)

In [54]:

import folium
import requests
from folium import plugins
from pprint import pprint
import json
from branca.element import Figure
import geocoder
from geopy.distance import distance
from openrouteservice import convert
import openrouteservice as ors

In [55]:
headers = {'User-Agent': 'mapsproject/1.0 (momen.saif22@gmail.com)'}
first_loc = input('Enter First Location (ex: giza, egypt):')
second_loc = input('Enter Second Location (ex: giza, egypt): ')
session = requests.Session()
session.headers.update(headers) 
address_1 = geocoder.osm(f'{first_loc}', session=session)
address_2 = geocoder.osm(f'{second_loc}', session=session)
coords1 = [address_1.lat, address_1.lng]
coords2 = [address_2.lat, address_2.lng]


In [56]:
def calculate_mid(coor1, coor2):
    mid_lat = (coor1[0] + coor2[0]) / 2
    mid_lon = (coor1[1] + coor2[1]) / 2
    return mid_lat, mid_lon

mid_lat, mid_lon = calculate_mid(coords1, coords2)


map_distance = distance(coords1, coords2).km
tile_zoom_level = 40075
zoom_level = 0
while tile_zoom_level > map_distance:
    tile_zoom_level = tile_zoom_level / 2
    zoom_level += 1

mymap = folium.Map(location=[mid_lat, mid_lon], zoom_start=zoom_level + 1, control_scale=True)

open_street = folium.TileLayer(tiles="OpenStreetMap", attr="Open Street map", name="Street Map").add_to(mymap)
folium.LayerControl().add_to(mymap)

mymap.add_child(plugins.MiniMap(toggle_display=True))
plugins.ScrollZoomToggler().add_to(mymap)
plugins.Fullscreen(position='topleft').add_to(mymap)

In [57]:
api_key = '5b3ce3597851110001cf62480a617482517d4d42af6cc9c5c3fc7113'
client = ors.Client(key=api_key)

base_url = 'https://api.openrouteservice.org/v2/directions/'

flip_coordinates = [[coords1[1], coords1[0]], [coords2[1], coords2[0]]]


payload = {"coordinates": flip_coordinates, "instructions": True}
headers = {"Authorization": api_key, "Content-Type": "application/json"}

mode = 'driving-car'
response = requests.post(f'{base_url}{mode}', json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    if "routes" in data and len(data["routes"]) > 0:
        distance = data['routes'][0]['summary']['distance']
        duration = data['routes'][0]['summary']['duration']
        route_geometry = data['routes'][0]['geometry']
        decoded_route = convert.decode_polyline(route_geometry)
        unflipped_route = [[coord[1], coord[0]] for coord in decoded_route['coordinates']]
        folium.PolyLine(unflipped_route, color='darkgreen').add_to(mymap)
    else:
        print("No routes found between the coordinates.")
else:
    print(f"API Error: {response.status_code}, Message: {response.text}")

start_name = data["routes"][0]["segments"][0]["steps"][0]["name"]
end_name = data["routes"][0]["segments"][0]["steps"][-1]["name"]
folium.Marker(coords1, popup=f'{address_1.lat}\n{address_1.lng}', tooltip=start_name, icon=folium.Icon(color='red', icon='s', prefix='fa')).add_to(mymap)
folium.Marker(coords2, popup=f'{address_2.lat}\n{address_2.lng}', tooltip=end_name, icon=folium.Icon(color='red', icon='e', prefix='fa')).add_to(mymap)

In [58]:
hours = int(duration // 3600)
minutes = int((duration % 3600) // 60)
formatted_duration = f"{hours:02d}h {minutes:02d}m"

# Output
print("Start Location:")
print(f"  Name: {start_name}")
print(f"  Coordinates: ({coords1[1]}, {coords1[0]})")

print("\nEnd Location:")
print(f"  Name: {end_name}")
print(f"  Coordinates: ({coords2[1]}, {coords2[0]})")

print(f"\nEstimated Duration: {formatted_duration}")
print(f"Calculated Distance: {round(distance / 1000, 2)}km")


Start Location:
  Name: -
  Coordinates: (34.295447, 27.8644422)

End Location:
  Name: -
  Coordinates: (32.897306, 24.091071)

Estimated Duration: 14h 42m
Calculated Distance: 1263.78km


In [59]:
fig = Figure(width=800, height=600)

fig.add_child(mymap)

mymap.save('map.html')
